In [1]:
import gpflow
import numpy as np
import math
import tensorflow as tf
sess = tf.Session()

In [8]:



class CustomKernel(gpflow.kernels.Kernel):
    def __init__(self,input_dim,variance=1.0, lengthscales=1.0,
                 active_dims=None, ARD=None, name=None):
        super().__init__(input_dim)
        self.sigma_u = gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  dtype=gpflow.settings.float_type)
        lengthscales, self.ARD = self._validate_ard_shape("lengthscales", lengthscales, ARD)
        self.lengthscales = gpflow.Param(lengthscales, transform=gpflow.transforms.positive,
                                      dtype=gpflow.settings.float_type)
        
        #tf.tensor([[1,2],[2,1]])
        #gpflow.Param(1.0, transform=gpflow.transforms.positive)
        #self.variance = gpflow.Param(1.0, transform=gpflow.transforms.positive)
    
    @gpflow.params_as_tensors
    def _scaled_square_dist(self, X, X2):
        """
        Returns ((X - X2ᵀ)/lengthscales)².
        Due to the implementation and floating-point imprecision, the
        result may actually be very slightly negative for entries very
        close to each other.
        """
        X = X / self.lengthscales
        Xs = tf.reduce_sum(tf.square(X), axis=-1, keepdims=True)

        if X2 is None:
            dist = -2 * tf.matmul(X, X, transpose_b=True)
            dist += Xs + tf.matrix_transpose(Xs)
            return dist

        X2 = X2 / self.lengthscales
        X2s = tf.reduce_sum(tf.square(X2), axis=-1, keepdims=True)
        dist = -2 * tf.matmul(X, X2, transpose_b=True)
        dist += Xs + tf.matrix_transpose(X2s)
        return dist
    
    @staticmethod
    def _clipped_sqrt(r2):
        # Clipping around the (single) float precision which is ~1e-45.
        return tf.sqrt(tf.maximum(r2, 1e-40))
    
    def scaled_square_dist(self, X, X2):  # pragma: no cover
        return self._scaled_square_dist(X, X2)
    
    @gpflow.params_as_tensors
    def K_r2(self, r2):
        return self.variance * tf.exp(-r2 / 2.)
    
    @gpflow.params_as_tensors
    def K(self, X, X2=None, presliced=False):
        
        #feature_vector_one = X[0:4]
        #time_index_one = X[4:5]
        #user_index_one = X[5:6]
        
        
        #if X2!=None:
            #feature_vector_two = X2[0:4]
            #time_index_two = X2[4:5]
            #user_index_two = X2[5:6]

        
        
        
        
        return self.sigma_u * 1
       
        
        #self.variance * tf.minimum(X, tf.transpose(X2))

    @gpflow.params_as_tensors
    def Kdiag(self, X, presliced=False):
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))

In [9]:
X = np.array([np.ones(6),np.zeros(6)])
y = np.array([[.3],[.5]])

In [14]:
    k =  CustomKernel(6)
    m = gpflow.models.GPR(X,y, kern=k)

In [15]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.312148
  Number of iterations: 5
  Number of functions evaluations: 9


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.312148
  Number of iterations: 5
  Number of functions evaluations: 9


In [66]:
#m.as_pandas_table()

In [22]:
def obtain_kernel_and_sigma_u(all_xs,all_ys):
    k =  CustomKernel(all_xs.shape[1])
    m = gpflow.models.GPR(all_xs, all_ys, kern=k)
    gpflow.train.ScipyOptimizer().minimize(m,session=sess)
    sigma_u = m.kern.sigma_u.value
    cov = np.zeros((all_xs.shape[0],all_xs.shape[0]))
    print(m.as_pandas_table())
    for i in range(all_xs.shape[0]):
        for j in range(all_xs.shape[0]):
            term = m.kern.K(all_xs[i],all_xs[j])
            cov[i][j]=term.eval(session=sess)
    return cov,sigma_u

In [23]:
obtain_kernel_and_sigma_u(X,y)


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.312148
  Number of iterations: 5
  Number of functions evaluations: 9


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.312148
  Number of iterations: 5
  Number of functions evaluations: 9


                             class prior transform  trainable shape  \
GPR/kern/lengthscales    Parameter  None       +ve       True    ()   
GPR/kern/sigma_u         Parameter  None       +ve       True    ()   
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape                 value  
GPR/kern/lengthscales           True                   1.0  
GPR/kern/sigma_u                True    0.1500026161960994  
GPR/likelihood/variance         True  0.020000046520069675  


(array([[0.15000262, 0.15000262],
        [0.15000262, 0.15000262]]), array(0.15000262))

In [118]:
#one = tf.constant([0,0])
#two = tf.constant([1,1])
inp= tf.constant([[1.0, 2, 3], [4, 5, 6.0], [7.0, 8, 9]])
tf.norm([inp,inp],ord=2)


<tf.Tensor 'norm_9/Squeeze:0' shape=() dtype=float32>

In [16]:

gr = m.kern.K(X,X)

In [19]:
X.shape

(2, 6)

In [20]:
gr.eval()

1.0